# ランキング学習（LGBMRanker）による勝率分析

## 本題
2024の有馬記念を予測する

# 結果
https://race.netkeiba.com/race/result.html?race_id=202406050811

In [1]:
import pandas as pd
import tqdm

import os
import sys
sys.path.append('../../')
import common.com as common
from common import const,utils
import copy
from datetime import datetime
import scraping.scraping as scraping


In [2]:
# DBにある全てのレースデータを重複を排除して取得（2024/12/29実行時点では2013-01_2024-12のデータ）
_data = common.get_race_data_distinct()
data=pd.DataFrame(_data,columns=const.ORIGIN_RACE_DATA_HEADERS)

# 有馬記念が含まれてしまっているので削除
data=data[data['レースID']!="202406050811"]

In [3]:
# ラベルエンコーディング(LabelEncoder)
from sklearn.preprocessing import LabelEncoder

encode_tar_cols = [
'性',
'斤量',
'芝かダートか',
'回り',
'馬場状態',
'天気',]

encoded = copy.deepcopy(data)

le = LabelEncoder()
for col in encode_tar_cols:
    encoded[col] = le.fit_transform(encoded[col].values)

In [4]:
# 着順関連度で重み付け
ORDER_RELATED = {1:30,2:28,3:26,4:24,5:22}
def get_order_related(order):
    try:
        return ORDER_RELATED[order]
    except KeyError:
        return 0
    
encoded['着順関連度'] = encoded['着順'].map(get_order_related)

In [5]:
encoded['オッズ'] = encoded['オッズ'].replace('---',0).astype(float)

In [6]:

# objectになってしまうので、floatに変換
for col in const.VARIABLE[:-1]:
    if encoded[col].dtype == 'object':
        if col in ['レースID','馬番','枠番','齢','馬のID','騎手のID','距離','競馬場ID']:
            encoded[col] = encoded[col].astype('int64')
        else:
            encoded[col] = encoded[col].astype(float)

In [7]:
# レースIDでグループ化
grouped_data = encoded.groupby('レースID')

In [8]:
import trueskill
mu = 25.
sigma = mu / 3.
beta = sigma / 2.
tau = sigma / 100.
draw_probability = 0.001
backend = None
env = trueskill.TrueSkill(
    mu=mu, sigma=sigma, beta=beta, tau=tau,
    draw_probability=draw_probability, backend=backend)


In [10]:
# レースごとにレーティング

uma_list = list(encoded.loc[:,'馬のID'].unique())
# 全馬のレートを初期化
all_horse_rate_dict = {k:env.create_rating() for k in uma_list}
rate_before_ = []
race_rate_ = []
race_after_ = []
result = pd.DataFrame(columns=encoded.columns.to_list().append(['直前レート','最新レート']))
i =0

for learning_count in range(10):
    print("learning_count:"+str(learning_count))
    for race_id,df_race in tqdm.tqdm(grouped_data):    
        rate_before = [env.expose(all_horse_rate_dict[el[8]]) for el in list(df_race.values)]
        # rate_mean = sum(rate_before)/len(rate_before)
        # チームを作成（1頭チーム x レースに出場する馬数）
        teams = [(all_horse_rate_dict[el[8]],) for el in list(df_race.values)]
        # レーティング
        teams = env.rate(teams, ranks=list(df_race['着順'].map(lambda x:x-1)))
        rate_after = [env.expose(t[0]) for t in teams]
        # race_rate = [(x-rate_mean)/(rate_mean + 0.001)*100 for x in rate_after]
        for i, el in enumerate(list(df_race.values)):
            # レートが高い方で更新
            all_horse_rate_dict[el[8]] = max([all_horse_rate_dict[el[8]],teams[i][0]])

        df_race['直前レート']=rate_before
        df_race['最新レート']=rate_after
        result=pd.concat([result,df_race])

learning_count:0


 35%|███▌      | 14577/41440 [08:25<15:31, 28.83it/s]


KeyboardInterrupt: 

In [66]:
# ランキング学習で馬を強い順に認識できているか確認
## 結果まったくできていないことが判明。ランキング学習の精度改善が必要

target = ['2021105898','2018105165','2020102781','2021105436','2019105552','2019105565','2019104447','2019105748','2019104740','2016104624']
data01 = result[['馬のID','馬の名前','直前レート','最新レート']]

arima = data01[data01['馬の名前']=='レガレイラ'].tail(1)
arima = pd.concat([arima,data01[data01['馬の名前']=='シャフリヤール'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='シャフリヤール'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='ダノンデサイル'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='ベラジオオペラ'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='ジャスティンパレス'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='アーバンシック'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='ローシャムパーク'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='スタニングローズ'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='ダノンベルーガ'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='シュトルーヴェ'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='プログノーシス'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='ブローザホーン'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='ディープボンド'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='スターズオンアース'].tail(1)])
arima = pd.concat([arima,data01[data01['馬の名前']=='ハヤヤッコ'].tail(1)])

arima.sort_values(by=['最新レート'],ascending=False)

KeyError: "None of [Index(['馬のID', '馬の名前', '直前レート', '最新レート'], dtype='object')] are in the [columns]"

In [ ]:
data01[data01['馬のID']==2021105898]


,馬のID,馬の名前,最新レート
514731,2021105898,レガレイラ,23.073146
527361,2021105898,レガレイラ,24.743118
537581,2021105898,レガレイラ,31.442044
537582,2021105898,レガレイラ,31.442043
558035,2021105898,レガレイラ,34.753861
552346,2021105898,レガレイラ,36.953850
584435,2021105898,レガレイラ,41.128994
571949,2021105898,レガレイラ,40.885869
579095,2021105898,レガレイラ,42.669799


In [12]:
# 8:2の割合で、学習用と評価用に分割
grouped_data_list = list(result.groupby('レースID'))
learn_count = round(len(grouped_data_list) * 0.8)

learn_grouped_data = grouped_data_list[:learn_count]
test_grouped_data = grouped_data_list[learn_count:]

# 8:2の割合で、訓練用と検証用に分割
train_count = round(len(learn_grouped_data) * 0.8)
train_grouped_data = learn_grouped_data[:train_count]
valid_grouped_data = learn_grouped_data[train_count:]

In [13]:
# 直前のレースまでのレートを特徴量に含めたうえで学習
target_variables=copy.deepcopy(const.VARIABLE)
target_variables.append('直前レート')
X_train = pd.concat([item[1].loc[:,target_variables] for item in train_grouped_data])
X_valid = pd.concat([item[1].loc[:,target_variables] for item in valid_grouped_data])
X_test = pd.concat([item[1].loc[:,target_variables] for item in test_grouped_data])
y_train = pd.concat([item[1].loc[:,'着順関連度'] for item in train_grouped_data])
y_valid = pd.concat([item[1].loc[:,'着順関連度'] for item in valid_grouped_data])
y_test = pd.concat([item[1].loc[:,'着順関連度'] for item in test_grouped_data])

In [14]:
# クエリーデータ(レース単位のデータ数のリスト)を作成
train_query = X_train.groupby('レースID').size().values.tolist()
valid_query = X_valid.groupby('レースID').size().values.tolist()

In [15]:
import lightgbm as lgb

model = lgb.LGBMRanker(
  random_state=100,              # 乱数シード
  n_estimators=500,              # 決定木の個数(default:100)
  learning_rate=0.01,            # 学習率(default:0.1)
  num_leaves=100,                 # 決定木にある分岐の個数(default:31)
  max_depth=-1,                  # 決定木の深さの最大値(default:-1)
  min_child_samples=150,         # 一つの葉に含まれる最小データ数(default:20) 
)

model.fit(X_train, y_train,
  group=train_query,             # 訓練用クエリーデータ
  eval_set=[(X_valid, y_valid)], # 学習時に用いる検証用データ
  eval_group=[valid_query],      # 検証用クエリーデータ
  eval_metric='ndcg',            # 学習時の評価手法
  eval_at=[1, 2, 3,4,5]              # 学習時の評価対象順位
)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028894 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1603
[LightGBM] [Info] Number of data points in the train set: 378923, number of used features: 20


LGBMRanker(learning_rate=0.01, min_child_samples=150, n_estimators=500,
           num_leaves=100, random_state=100)

In [16]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration_)

# 予測結果や関連度をDataFrameに連結
df_pred = pd.DataFrame({
    'レースID': X_test['レースID'],
    '馬番号': X_test['馬のID'],
    '予想スコア':  y_pred,
    '着順関連度': y_test,
})

In [17]:
df_pred.to_csv(const.BASE_DIR+'predict//LGBMRanker02//predict.csv')

In [18]:
# 特徴量重要度の抽出
df_importances = pd.DataFrame({'columns':X_train.columns, 'importances':model.feature_importances_})
df_importances.sort_values('importances', ascending=False, inplace=True)
print(df_importances)

   columns  importances
2       馬番        14045
1       枠番        10797
17   競馬場ID         4253
13      距離         3841
12  芝かダートか         3675
4        齢         2698
18   着順関連度         2500
9       人気         2032
0    レースID         1337
8      オッズ         1283
19   直前レート         1002
6     馬のID          683
14      回り          401
5       斤量          286
7    騎手のID          242
10      体重          211
3        性          113
11    体重変化           75
16      天気           24
15    馬場状態            2


In [19]:
from sklearn.metrics import ndcg_score
# クエリーごとにNDCGを計算し、その平均値を算出
ndcg_score = df_pred.groupby('レースID').apply(lambda d: ndcg_score([d['着順関連度']], [d['予想スコア']], k=5)).mean()
print(ndcg_score)

1.0


C:\Users\ionde\AppData\Local\Temp\ipykernel_15916\245059073.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_score = df_pred.groupby('レースID').apply(lambda d: ndcg_score([d['着順関連度']], [d['予想スコア']], k=5)).mean()


In [20]:
import pickle
# モデルの保存
file_dir = const.BASE_DIR+'predict//LGBMRanker02//LGBMRanker02.pickle'
pickle.dump(model, open(file_dir, 'wb'))

In [21]:
check=result.groupby('馬のID')
sorted_by_rate = copy.deepcopy(result.loc[check['日付'].idxmax(),:])
sorted_by_rate[['馬の名前','最新レート']].sort_values(by='最新レート',ascending=False).head(50)

,馬の名前,最新レート
437025,グランアレグリア,54.330175
390976,モズアスコット,53.761015
488297,ダイアトニック,53.600164
389965,アーモンドアイ,53.433255
437028,インディチャンプ,53.390205
580143,ソウルラッシュ,52.932686
350255,コパノキッキング,52.841726
390973,ゴールドドリーム,52.597418
581661,レモンポップ,52.479721
438538,チュウワウィザード,52.314052


In [22]:
sorted_by_rate

,レースID,着順,枠番,馬番,馬の名前,性,齢,斤量,馬のID,騎手の名前,...,芝かダートか,距離,回り,馬場状態,天気,競馬場ID,競馬場名,着順関連度,直前レート,最新レート
49063,201306050710,11,6,12,スプリングゲント,2,1,27,2000100231,白浜雄造,...,2,4100,3,1,1,6,中山,0,0.000000,18.650633
49068,201306050710,99,6,11,メルシーエイタイム,2,1,27,2002104783,横山義行,...,2,4100,3,1,1,6,中山,0,0.000000,0.956241
50731,201408010211,4,5,5,トウカイトリック,2,1,17,2002107238,福永祐一,...,1,3000,0,2,1,8,京都,24,27.358611,29.619854
16426,201308030304,11,4,4,ガブリン,2,1,23,2002110227,平沢健治,...,2,2910,3,2,1,8,京都,0,15.087863,14.130947
6648,201305010810,11,7,9,セタガヤフラッグ,2,1,8,2003100055,左海誠二,...,1,2000,1,2,1,5,東京,0,0.000000,0.427519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584940,202408070906,12,3,3,シーソーゲーム,2,2,15,2022110145,Ｍ．デム,...,0,1800,0,2,1,8,京都,0,21.402471,21.679499
583219,202408070503,11,4,6,ゲッティヴィラ,2,2,15,2022110151,松山弘平,...,1,1200,0,2,2,8,京都,0,21.025645,20.734964
579499,202408060503,6,4,5,レーティッシュ,2,2,15,2022110152,Ｃ．デム,...,0,1400,0,2,2,8,京都,0,18.018528,22.366794
582400,202406050407,2,7,10,ベリタバグス,2,2,15,2022110153,黛弘人,...,0,1800,0,2,1,6,中山,28,26.844466,31.275250
